In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
class AdvancedPortfolioManagement:
    def __init__(self, returns, cov_matrix, market_caps):
        """
        Initialize the class with returns, covariance matrix, and market capitalizations of assets.
        
        :param returns: Expected returns of assets
        :param cov_matrix: Covariance matrix of asset returns
        :param market_caps: Market capitalizations of assets
        """
        self.returns = returns
        self.cov_matrix = cov_matrix
        self.market_caps = market_caps
        self.market_weights = market_caps / np.sum(market_caps)
        self.tau = 0.05  # Scalar factor for prior uncertainty

    def factor_investing(self, factors):
        """
        Factor investing based on given factors.
        
        :param factors: DataFrame with factor loadings for each asset
        :return: Factor-based portfolio weights
        """
        factor_weights = factors.mean(axis=0)
        normalized_weights = factor_weights / np.sum(factor_weights)
        return normalized_weights
    
    def risk_parity(self):
        """
        Risk parity portfolio construction.
        
        :return: Risk parity portfolio weights
        """
        inv_vol = 1 / np.sqrt(np.diag(self.cov_matrix))
        risk_parity_weights = inv_vol / np.sum(inv_vol)
        return risk_parity_weights

    def black_litterman(self, P, Q, omega):
        """
        Black-Litterman model for portfolio optimization.
        
        :param P: Pick matrix for views
        :param Q: View returns
        :param omega: Uncertainty in views
        :return: Black-Litterman adjusted expected returns and portfolio weights
        """
        # Step 1: Market equilibrium returns (implied equilibrium excess returns)
        pi = self.tau * (self.cov_matrix @ self.market_weights)
        
        # Step 2: Adjust the prior returns with the views
        M_inverse = np.linalg.inv((P @ (self.tau * self.cov_matrix) @ P.T) + omega)
        middle_term = (self.tau * self.cov_matrix) @ P.T @ M_inverse
        adjusted_returns = pi + middle_term @ (Q - (P @ pi))
        
        # Step 3: Optimize portfolio with adjusted returns
        weights = self.mean_variance_optimization(adjusted_returns)
        return adjusted_returns, weights

    def mean_variance_optimization(self, expected_returns):
        """
        Mean-variance optimization to find the portfolio with the highest Sharpe ratio.
        
        :param expected_returns: Expected returns of assets
        :return: Optimized portfolio weights
        """
        num_assets = len(expected_returns)
        
        def portfolio_performance(weights):
            port_return = weights @ expected_returns
            port_volatility = np.sqrt(weights.T @ self.cov_matrix @ weights)
            return -port_return / port_volatility
        
        constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
        bounds = [(0, 1) for _ in range(num_assets)]
        initial_guess = np.full(num_assets, 1. / num_assets)
        
        result = minimize(portfolio_performance, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)
        return result.x

